In [2]:
import pandas as pd

In [111]:
PPC_BOM_PATH = 'ppc_bom.xlsx'
PPC_BOM_SHEET = 'ReportUnitCost20221212'
PPC_CLEANED_BOM_PATH = 'ppc_bom_cleaned.csv'

In [84]:
ppc_bom = pd.read_excel(PPC_BOM_PATH, sheet_name=PPC_BOM_SHEET, header = 2, usecols="B:Z,AK,BI")
# ppc_bom_small = pd.read_excel(PPC_BOM_PATH, sheet_name=PPC_BOM_SHEET, header = 2, usecols="B:C,H,J,L,M,P,R,T,V:X,AK")

In [85]:
ppc_bom.columns = ppc_bom.columns.str.replace(r'\n', ' ', regex=True)
# ppc_bom.rename(columns={'SYS PARTS/ CAR': 'Quantity', 'PARTNO': 'PartNumber', } )
ppc_bom.head()

,COST CARR,D1,D1 TITLE,Product Stream,D2,D2 TITLE,SU,SU TITLE,KU,KU TITLE,KU STATUS,RESP DEPT,PSSNO,PSS NAME,ART no,ART name,CMNO,CM NAME,FGRP,FGRP NAME,POS,PARTNO,PARTNO SUFF,PART NAME,DONOR,SYS PARTS/ CAR,Product Module
0,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,,,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,700,ART EXTERIOR FRONT & REAR,00700,EXTERIOR FRONT AND REAR,2,P8 US Inscription 7 seats,8210,BONNET,050,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,4.0,Exterior Front & Rear
1,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,,,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,700,ART EXTERIOR FRONT & REAR,00700,EXTERIOR FRONT AND REAR,2,P8 US Inscription 7 seats,8250,FRONT FENDER,010,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,12.0,Exterior Front & Rear
2,MEP2P,292253,ELECTRIC PROPULSION,ELECTRIC PROPULSION,,,292300,THERMAL MANAGEMENT,302178,Cooling system V536,DECID,97400,830,ART THERMAL MANAGEMENT,00830,THERMAL MANAGEMENT,2,P8 US Inscription 7 seats,2681,,050,30622624,000,RETAINER 22,MBOM,1.0,Thermal Exchange & Distribution
3,V536X,292248,SOFTWARE AND ELECTRONICS PLATFORM,SOFTWARE AND ELECTRONICS PLATFORM,,,292271,CORE SYSTEM COMPONENTS,292340,Core system components,DECID,94200,590,ART CORE SYSTEM COMPONENTS,00590,CORE SYSTEM COMPONENTS,2,P8 US Inscription 7 seats,3725,,050,30622876,000,SIX POINT SOCKET SCREW M5,MBOM,3.0,Computing Infrastructure
4,V536X,292251,VEHICLE PLATFORM,VEHICLE PLATFORM,,,292292,BODY CONTROL AND ROAD CONTACT,280313,Tuning & control,DECID,92520,640,ART BODY CONTROL & ROAD CONTACT,00640,BODY CONTROL AND ROAD CONTACT,2,P8 US Inscription 7 seats,7612,SHOCK ABSORBER,010,30622916,000,SEMS SCREW M8*30*MATHREAD,MBOM,4.0,Body Control


In [86]:
ppc_bom.dtypes

COST CARR          object
D1                  int64
D1 TITLE           object
Product Stream     object
D2                 object
D2 TITLE           object
SU                  int64
SU TITLE           object
KU                  int64
KU TITLE           object
KU STATUS          object
RESP DEPT           int64
PSSNO               int64
PSS NAME           object
ART no             object
ART name           object
CMNO                int64
CM NAME            object
FGRP               object
FGRP NAME          object
POS                object
PARTNO              int64
PARTNO SUFF        object
PART NAME          object
DONOR              object
SYS PARTS/ CAR    float64
Product Module     object
dtype: object

In [109]:
print(f'Number of rows before clean: {ppc_bom.index.stop}')
# Remove Zero Quantity values
ppc_bom_cleaned = pd.DataFrame(ppc_bom[ppc_bom['SYS PARTS/ CAR'] != 0.0])
# cleanded_ppc_bom =ppc_bom.drop(rows_to_remove)

# print(f'Number of rows after clean: {cleanded_ppc_bom.index}')
ppc_bom_cleaned
print(f'Number of rows after clean: {ppc_bom_cleaned.index}')


Number of rows before clean: 21969
Number of rows after clean: Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            17079, 17080, 17081, 17082, 17083, 17084, 17085, 17086, 17087,
            17088],
           dtype='int64', length=7002)


In [112]:
ppc_bom_cleaned.to_csv(PPC_CLEANED_BOM_PATH)